In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, regularizers, optimizers
import datetime
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [2]:
#define constants
EPOCHS = 50
BATCH_SIZE = 64
VERBOSE = 1
OPTIM = tf.keras.optimizers.RMSprop()

In [3]:
#image characteristics (CIFAR_!= is a set of 60k images 32x32 pixels on 3 channels)
IMG_CHANNELS, IMG_ROWS, IMG_COLS = 3, 32, 32 #input image dimensions
INPUT_SHAPE = (IMG_ROWS, IMG_COLS, IMG_CHANNELS)
NUM_CLASSES = 10

In [4]:
def load_data():
    #load data
    (X_train, y_train), (X_test, y_test) = datasets.cifar10.load_data()
    #cast
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    #Standardization
    mean = np.mean(X_train, axis =(0,1,2,3))
    std = np.std(X_train, axis =(0,1,2,3))
    eps = 1e-7
    X_train = (X_train - mean) / (std + eps)
    X_test = (X_test - mean) / (std + eps)
    #convert class vectors to binary class matrices
    y_train = tf.keras.utils.to_categorical(y_train, NUM_CLASSES)
    y_test = tf.keras.utils.to_categorical(y_test, NUM_CLASSES)
    
    return X_train, X_test, y_train, y_test

In [5]:
#load data
X_train, X_test, y_train, y_test = load_data()

In [6]:
#let's define a convnet with the following modules:
# 1st --> (CONV + CONV + MaxPool + DropOut)
# 2nd --> (CONV + CONV + MaxPool + DropOut)
# 3rd --> (CONV + CONV + MaxPool + DropOut)
# dense --> (FLATTEN + SOFTMAX classifier)

def build_model(input_shape, classes):
    model = models.Sequential()
    #1st module
    model.add(layers.Conv2D(32, (3,3), padding = 'same', activation = 'relu', input_shape = input_shape))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(32, (3,3), padding = 'same', activation = 'relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size = (2,2)))
    model.add(layers.Dropout(0.2))
    #2nd module
    model.add(layers.Conv2D(64, (3,3), padding = 'same', activation = 'relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(64, (3,3), padding = 'same', activation = 'relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size = (2,2)))
    model.add(layers.Dropout(0.3))
    #3rd module
    model.add(layers.Conv2D(128, (3,3), padding = 'same', activation = 'relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(128, (3,3), padding = 'same', activation = 'relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size = (2,2)))
    model.add(layers.Dropout(0.4))
    #dense
    model.add(layers.Flatten())
    model.add(layers.Dense(classes, activation = "softmax"))
    
    return model

In [7]:
#TensorBoard
LOG_DIR = "logs/CNN_cifar10_dataaug/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
#tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=LOG_DIR, histogram_freq=1)
callbacks = [tf.keras.callbacks.TensorBoard(log_dir = LOG_DIR)]

In [8]:
#initialize the optimizer and the model
model = build_model(input_shape = INPUT_SHAPE, classes = NUM_CLASSES)
model.compile(loss = "categorical_crossentropy", optimizer = OPTIM, metrics = ["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        1

In [9]:
#data augmentation
datagen = ImageDataGenerator(
        rotation_range = 30,
        width_shift_range = 0.2,
        height_shift_range = 0.2,
        horizontal_flip = True)
datagen.fit(X_train)

In [ ]:
#fit the model and evaluate
history = model.fit_generator(datagen.flow(X_train, y_train, batch_size = BATCH_SIZE), epochs = EPOCHS, verbose = VERBOSE, validation_data = (X_test, y_test), callbacks = callbacks)
score = model.evaluate(X_test, y_test, batch_size = BATCH_SIZE, verbose = VERBOSE)
print("\nTest score:", score[0])
print("Test accuracy:", score[1])

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/50
  1/782 [..............................] - ETA: 0s - loss: 4.6616 - accuracy: 0.1250WARNING:tensorflow:From C:\Users\ER180124\.conda\envs\PrognosticEnv\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
782/782 [==============================] - 368s 470ms/step - loss: 2.0958 - accuracy: 0.3638 - val_loss: 1.5703 - val_accuracy: 0.4833
Epoch 2/50
782/782 [==============================] - 365s 467ms/step - loss: 1.5668 - accuracy: 0.4979 - val_loss: 1.4752 - val_accuracy: 0.5361
Epoch 3/50
782/782 [==============================] - 345s 442ms/step - loss: 1.3826 - accuracy: 0.5558 - val_loss: 1.1879 - val_accuracy: 0.6235
Epoch 4/50
782/782 [==============================] - 341s 437ms/step - loss: 1.2444 - accuracy: 0.58

In [ ]:
%load_ext tensorboard
%tensorboard --logdir LOG_DIR

In [ ]:
#save model to disk
model_json = model.to_json()
with open('cifar10_model.json','w') as json_file:
    json_file.write(model_json)
model.save_weights('cifar10_weights.h5')

In [ ]:
#predicting without training each time
from tensorflow.keras.models import model_from_json
model = model_from_json(open('cifar10_model.json').read())
model.load_weights('cifar10_weights.h5')

In [ ]:
#load images to be predicted
from imageio import imread
from skimage.transform import resize
img_names = ['cat-standing.jpg','dog.jpg']
imgs = [resize(imread(img_name), (32,32)).astype("float32") for img_name in img_names]
imgs = np.array(imgs) / 255
print("imgs.shape: ",imgs.shape)

In [ ]:
#compile the model
model.compile(loss = 'categorical_crossentropy', optimizer = OPTIM, metrics = ['accuracy'])
#predict
predictions =np.argmax(model.predict(imgs), axis=-1)
print("predictions:", predictions)